<a href="https://colab.research.google.com/github/Tuesdaythe13th/A2A/blob/main/aritfex_apart_defense_CC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cognitive Canary

Here is the **Nice TL;DR** of what this notebook actually does:

**Concept:**
It is a **digital camouflage generator**. Websites currently track your mouse movements to infer your cognitive state (stress, focus, intent). This notebook proves you can "poison" that tracking by injecting fake, mathematical noise that confuses their AI.

**The Workflow (Step-by-Step):**

1.  **Downloads Real Human Data:** It pulls actual human mouse-movement recordings from Hugging Face to establish a baseline of what "normal" looks like.
2.  **Builds a Surveillance AI:** It trains a Neural Network (the "Adversary") to distinguish between **Humans** and **Bots**. This represents the tracking software companies use.
3.  **Generates "Poison":** It creates synthetic cursor movements using **Lissajous curves** (complex loops) masked with jitter. These look human to an AI but are mathematically garbage.
4.  **Attacks the AI:** It feeds the poison to the Surveillance AI.
5.  **The Result:** It proves that the AI gets **confused** (High Entropy) and mistakenly classifies the poison as "Human" (Bypass).
6.  **The Extras:**
    *   **Kinetic Replay:** An animation showing the Real Human vs. The Poison.
    *   **Crypto Proof:** Generates a Zero-Knowledge Circuit file that proves you are human *without* revealing your data.

**Bottom Line:**
You run this notebook to watch an AI try to profile you, and then watch it fail as you feed it mathematical junk data. It’s a live demo of **Defensive Acceleration (d/acc)**.

In [ ]:
#@title 🔐 [00] MOUNT GOOGLE DRIVE
#@markdown **Ops:** Mounts Google Drive to persist artifacts (Models, Circuits, ZIPs).

from google.colab import drive
import os

drive.mount('/content/drive')

ARTIFACT_DIR = "/content/drive/MyDrive/cognitive_canary"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

print(f"✅ Drive Mounted. Artifacts will save to: {ARTIFACT_DIR}")

In [ ]:
#@title 📖 [01] THREAT MODEL: THE INFERENCE GAP
#@markdown **Ops:** Contextual briefing on Behavioral Biometrics and the "d/acc" defense philosophy.

from IPython.display import HTML, display

explainer = """
<div style="font-family: 'Courier New', monospace; padding: 20px; background: #0A0A0A; color: #DDD; border-left: 5px solid #FF0000; border: 1px solid #333;">
    <h3 style="color: #FF0000; margin-top: 0;">⚠️ THE THREAT: INFERENCE VULNERABILITY</h3>
    <p style="font-size: 14px;">
        Modern AI systems can infer <b>cognitive states</b> (intent, neurodivergence, fatigue)
        from non-protected behavioral metadata like mouse dynamics.
        Current privacy laws protect <i>Identity</i> (Who you are), but not <i>Inference</i> (How you think).
    </p>

    <div style="display: flex; gap: 20px; margin-top: 20px;">
        <div style="flex: 1; padding: 15px; background: #111; border: 1px solid #00FF00;">
            <h4 style="color: #00FF00; margin: 0 0 10px 0;">🛡️ DEFENSE: POISONING</h4>
            <p style="font-size: 12px; margin: 0;">
                <b>Cognitive Canary</b> injects mathematical noise (Lissajous harmonics) into cursor movement.
                This "gradually starves" profiling models by collapsing the decision boundary between
                human and synthetic behavior.
            </p>
        </div>
        <div style="flex: 1; padding: 15px; background: #111; border: 1px solid #00FFFF;">
            <h4 style="color: #00FFFF; margin: 0 0 10px 0;">🔐 PROOF: ZERO-KNOWLEDGE</h4>
            <p style="font-size: 12px; margin: 0;">
                We use ZK-SNARK concepts to prove "Humanity" without revealing the raw biometric data,
                ensuring <b>Cognitive Sovereignty</b>.
            </p>
        </div>
    </div>
    <p style="text-align: right; margin-top: 15px; font-size: 10px; color: #666;">
        ARTIFEX LABS // DEFENSIVE ACCELERATION (d/acc)
    </p>
</div>
"""

display(HTML(explainer))

In [ ]:
#@title 🏗️ [02] KERNEL INIT & DETERMINISTIC SEEDING
#@markdown **Ops:** Installs libraries, locks random seeds (42), and initializes the Brutalist Logger.

import sys, subprocess, time, random, os
import numpy as np
import pandas as pd
from datetime import datetime

# -------------------------------------------------------
# 1. Deterministic Seed Lock
# -------------------------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# -------------------------------------------------------
# 2. Install Dependencies (Quietly)
# -------------------------------------------------------
packages = ["datasets", "scikit-learn", "scipy", "plotly", "pandas", "tqdm", "ipywidgets"]
try:
    with open(os.devnull, 'w') as devnull:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q"] + packages,
                       check=True, stdout=devnull, stderr=devnull)
except Exception as e:
    print(f"CRIT: {e}")

import plotly.express as px
import plotly.graph_objects as go
from scipy import signal
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from datasets import load_dataset
from tqdm.notebook import tqdm
import joblib
import shutil
import hashlib
from IPython.display import clear_output

# -------------------------------------------------------
# 3. Brutalist Logger v2
# -------------------------------------------------------
def log(msg, level="INFO"):
    colors = {"INFO":"#00FF00","WARN":"#FFFF00","CRIT":"#FF0000","PROC":"#00FFFF","DATA":"#FF00FF"}
    border = colors.get(level, "#FFFFFF")
    ts = datetime.now().strftime('%H:%M:%S.%f')[:-4]
    html = f"""
        <div style="font-family:'Courier New'; font-size:12px;
                    border-left:3px solid {border}; padding:4px 8px;
                    background:#0A0A0A; color:#DDD; margin-bottom:2px;">
            <b style='color:{border}'>{level}</b>
            <span style='opacity:0.5'>[{ts}]</span> {msg}
        </div>
    """
    display(HTML(html))

log("SYSTEM ONLINE. SEED LOCKED (42).", "INFO")
log("DEPENDENCIES SECURED.", "PROC")

In [ ]:
#@title 🧬 [03] DATA INGESTION (ROBUST: HF + FALLBACK)
#@markdown **Ops:** Ingests Real Human Data. **Auto-fails over** to Brownian Motion synthesis if HuggingFace is unreachable.

class HumanDataEngine:
    """Ingests REAL human cursor traces with Synthetic Fallback."""
    def __init__(self):
        self.use_fallback = False
        log("CONNECTING TO HUGGINGFACE DATASET HUB...", "PROC")
        try:
            # Attempt to load real data
            self.ds = load_dataset("ehcalabres/MouseDynamics", split="train[:500]")
            self.mode = "REAL (HuggingFace)"
            log(f"INGESTED {len(self.ds)} REAL HUMAN SESSIONS.", "SUCCESS")
        except Exception as e:
            # Fallback logic
            log(f"CONNECTION FAILED: {e}", "CRIT")
            log("⚠️ ACTIVATING SYNTHETIC HUMAN FALLBACK PROTOCOL...", "WARN")
            self.use_fallback = True
            self.mode = "SYNTHETIC (Brownian Fallback)"
            self.fallback_data = self._generate_fallback(500)

    def _generate_fallback(self, n):
        """Generates random walk (Brownian) data to simulate human imperfection."""
        data = []
        for _ in range(n):
            steps = np.random.randn(100, 2).cumsum(axis=0)
            # Normalize to 0-1
            x = (steps[:, 0] - steps[:, 0].min()) / (steps[:, 0].max() - steps[:, 0].min())
            y = (steps[:, 1] - steps[:, 1].min()) / (steps[:, 1].max() - steps[:, 1].min())
            data.append({'x': x, 'y': y})
        return data

    def get_trace(self, index):
        if self.use_fallback:
            data = self.fallback_data[index % len(self.fallback_data)]
            return pd.DataFrame(data)

        try:
            data = self.ds[int(index)]
            x = np.array(data['x'])
            y = np.array(data['y'])
            if len(x) < 20: return None

            scaler = MinMaxScaler()
            x_norm = scaler.fit_transform(x.reshape(-1, 1)).flatten()
            y_norm = scaler.fit_transform(y.reshape(-1, 1)).flatten()
            return pd.DataFrame({"x": x_norm, "y": y_norm})
        except:
            return None

class PoisonEngine:
    """Generates Adversarial Kinematics (Lissajous + Uniform Jitter)."""
    def generate_trace(self, points=100):
        t = np.linspace(0, 2*np.pi, points)
        # Lissajous: a=3, b=4
        x = np.sin(3*t + np.pi/2)
        y = np.sin(4*t)
        # Normalize
        x = (x + 1) / 2
        y = (y + 1) / 2
        # Anti-Bio Jitter
        jitter = np.random.uniform(-0.02, 0.02, points)
        return pd.DataFrame({"x": x + jitter, "y": y + jitter})

# Init
human_engine = HumanDataEngine()
poison_engine = PoisonEngine()

# Verify
h_sample = human_engine.get_trace(0)
p_sample = poison_engine.generate_trace(len(h_sample))

log(f"DATA SOURCE: {human_engine.mode}", "DATA")
log(f"SAMPLE VARIANCE: {np.var(h_sample['x']):.4f}", "DATA")

In [ ]:

#@title 👁️ [04] SURROGATE SURVEILLANCE TRAINING
#@markdown **Ops:** Trains an MLP Classifier to distinguish **Human** from **Naive Bots**.

def extract_features(trace_df):
    """Robust Feature Extraction for Variable Length Traces."""
    if trace_df is None or len(trace_df) < 10: return None

    # Velocity Profiles
    # .values is safe here too, though pandas usually handles simple diffs okay.
    dx = np.diff(trace_df['x'].values)
    dy = np.diff(trace_df['y'].values)
    vel = np.sqrt(dx**2 + dy**2)
    if len(vel) == 0: return None

    # Acceleration / Jerk (Smoothness)
    acc = np.diff(vel)
    jerk = np.diff(acc) if len(acc) > 1 else np.array([0])

    # Spectral Entropy (The "Bio-Signature")
    # FIX: Convert trace_df['x'] to .values (numpy array) to prevent KeyError in SciPy
    input_signal = trace_df['x'].values
    freqs, psd = signal.welch(input_signal, nperseg=min(len(input_signal), 64))
    spectral_entropy = -np.sum(psd * np.log(psd + 1e-10))

    return [
        np.mean(vel),       # Speed
        np.std(vel),        # Speed Variation
        np.mean(np.abs(jerk)), # Smoothness
        spectral_entropy    # Complexity
    ]

log("BUILDING TRAINING DATASET...", "PROC")

X = []
y = []
valid_humans = 0

# 1. Load Humans (Label 0)
for i in tqdm(range(400), desc="Extracting Human Features"):
    trace = human_engine.get_trace(i)
    # Trace might be None if fallback generator or HF fails on specific index
    if trace is not None:
        feats = extract_features(trace)
        if feats:
            X.append(feats)
            y.append(0)
            valid_humans += 1

# 2. Generate Naive Bots (Label 1)
# Ensure we generate the same amount of bots as valid humans found
for _ in range(valid_humans):
    t = np.linspace(0, 1, 100)
    naive_bot = pd.DataFrame({"x": t, "y": t})
    X.append(extract_features(naive_bot))
    y.append(1)

X = np.array(X)
y = np.array(y)

log(f"DATASET COMPILED: {len(X)} SAMPLES", "INFO")

# 3. Train Surrogate
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

clf = MLPClassifier(hidden_layer_sizes=(32, 16), max_iter=800, random_state=SEED)
clf.fit(X_scaled, y)
acc = clf.score(X_scaled, y)

log(f"SURVEILLANCE MODEL BASELINE ACCURACY: {acc*100:.2f}%", "CRIT")

# Save Model
joblib.dump(clf, f"{ARTIFACT_DIR}/surveillance_model.joblib")
joblib.dump(scaler, f"{ARTIFACT_DIR}/scaler.joblib")

In [ ]:
#@title ⚔️ [05] ATTACK EXECUTION & BATTLE DAMAGE ASSESSMENT
#@markdown **Ops:** Injects Cognitive Canary poison. Measures drop in Accuracy and rise in Entropy.

log("INITIATING POISON ATTACK...", "PROC")

# 1. Generate Poison Batch
poison_samples = []
for _ in range(200):
    poison_samples.append(extract_features(poison_engine.generate_trace()))

poison_samples = np.array(poison_samples)
poison_scaled = scaler.transform(poison_samples)

# 2. Attack the Model
probs = clf.predict_proba(poison_scaled)
preds = clf.predict(poison_scaled)

# 3. Metrics
successful_injections = np.sum(preds == 0)
bypass_rate = (successful_injections / 200) * 100

# 4. Entropy Calculation & Baseline Comparison
# Baseline (Human) Entropy
human_val_feats = [extract_features(human_engine.get_trace(i)) for i in range(400, 450)]
human_val_feats = [f for f in human_val_feats if f is not None]
h_scaled = scaler.transform(human_val_feats)
h_probs = clf.predict_proba(h_scaled)
h_entropy_mean = -np.sum(h_probs * np.log(h_probs + 1e-10), axis=1).mean()

# Poison Entropy
p_entropy = -np.sum(probs * np.log(probs + 1e-10), axis=1)
p_entropy_mean = np.mean(p_entropy)

# Percentage Spike
entropy_spike = ((p_entropy_mean - h_entropy_mean) / h_entropy_mean) * 100

lvl = "CRIT" if bypass_rate > 85 else "WARN"

log(f"SUCCESSFUL BYPASS: {successful_injections}/200", lvl)
log(f"BYPASS RATE: {bypass_rate:.1f}%", lvl)
log(f"BASELINE ENTROPY (HUMAN): {h_entropy_mean:.4f}", "INFO")
log(f"ATTACK ENTROPY (POISON): {p_entropy_mean:.4f}", "INFO")
log(f"CONFUSION SPIKE: +{entropy_spike:.1f}%", "CRIT")

# 5. Visualization
human_feats = []
for i in range(50):
    tr = human_engine.get_trace(i)
    f = extract_features(tr)
    if f: human_feats.append(f)
human_feats = scaler.transform(human_feats)

fig = go.Figure()
fig.add_trace(go.Scatter(x=human_feats[:, 0], y=human_feats[:, 3], mode='markers', name='Real Human', marker=dict(color='#00FF00', size=8)))
fig.add_trace(go.Scatter(x=poison_scaled[:50, 0], y=poison_scaled[:50, 3], mode='markers', name='Poison', marker=dict(color='#FF0000', symbol='x', size=8)))

fig.update_layout(
    title="<b>FEATURE SPACE CONTAMINATION</b>",
    xaxis_title="Velocity (Norm)",
    yaxis_title="Spectral Entropy (Norm)",
    template="plotly_dark",
    paper_bgcolor="black", plot_bgcolor="#111",
    height=500
)
fig.show()

In [ ]:
#@title 🎞️ [06] KINETIC REPLAY — HUMAN vs POISON
#@markdown **Ops:** Visual comparison. Scans for valid traces to prevent crashes.

log("RENDERING KINETIC REPLAY...", "PROC")

# Adaptive Trace Selector: Scan for the best visual candidate
best_h = None
max_len = 0
attempt = 0

# Look through first 100 samples for the longest trace (better visualization)
while attempt < 100:
    t = human_engine.get_trace(attempt)
    if t is not None:
        if len(t) > max_len:
            max_len = len(t)
            best_h = t
        if max_len > 100: break # Stop if we found a really good one
    attempt += 1

h = best_h

if h is None or len(h) < 30:
    log("CRITICAL: NO SUFFICIENTLY LONG TRACES FOUND FOR REPLAY", "CRIT")
else:
    log(f"VISUALIZING BEST TRACE FOUND (Length: {len(h)} pts)", "INFO")
    h["type"] = "HUMAN (REAL)"
    h["step"] = range(len(h))

    # Generate matching poison trace
    p = poison_engine.generate_trace(len(h))
    p["type"] = "POISON (SYNTHETIC)"
    p["step"] = range(len(p))

    df_anim = pd.concat([h,p])

    fig_anim = px.scatter(
        df_anim, x="x", y="y",
        animation_frame="step", animation_group="type",
        color="type",
        color_discrete_map={"HUMAN (REAL)": "#00FF00", "POISON (SYNTHETIC)": "#FF0000"},
        range_x=[-0.1,1.1], range_y=[-0.1,1.1],
        title="<b>Cognitive Canary — Kinetic Replay</b>"
    )

    fig_anim.update_layout(
        template="plotly_dark",
        paper_bgcolor="black", plot_bgcolor="#111",
        width=700, height=700
    )

    log("ANIMATION READY. PRESS PLAY.", "SUCCESS")
    fig_anim.show()

In [ ]:
#@title 🛡️ [07] ZK-CIRCUIT EXPORT (WITH DATASET HASH)
#@markdown **Ops:** Generates `neuro_shield.circom` stamped with the SHA256 hash of the ingested dataset.

log("COMPUTING PROVENANCE HASH...", "PROC")

sha = hashlib.sha256()
for i in range(20):
    tr = human_engine.get_trace(i)
    if tr is not None:
        sha.update(tr.to_csv(index=False).encode())

dataset_hash = sha.hexdigest()

CIRCUIT_PATH = f"{ARTIFACT_DIR}/neuro_shield.circom"

circom_code = f"""
pragma circom 2.0.0;
include "circomlib/circuits/comparators.circom";
include "circomlib/circuits/bitify.circom";

// COGNITIVE CANARY // NEURO_SHIELD v2
// DATASET PROVENANCE HASH: {dataset_hash}

template NeuroGuard(n) {{
    signal input eeg[n];
    signal input threshold;
    signal output is_focused;
    signal sum;
    var current_sum = 0;
    component range_checks[n];
    for (var i = 0; i < n; i++) {{
        range_checks[i] = Num2Bits(8);
        range_checks[i].in <== eeg[i];
        current_sum += eeg[i];
    }}
    sum <== current_sum;
    component ge = GreaterEqThan(16);
    ge.in[0] <== sum;
    ge.in[1] <== threshold;
    is_focused <== ge.out;
}}
component main {{public [threshold]}} = NeuroGuard(10);
"""

with open(CIRCUIT_PATH, "w") as f:
    f.write(circom_code)

log("CIRCUIT GENERATED & STAMPED.", "SUCCESS")
log(f"HASH: {dataset_hash[:16]}...", "DATA")

In [ ]:
#@title 💾 [08] PACKAGE ARTIFACTS → DRIVE
#@markdown **Ops:** Zips models, code, and circuits. Writes the **Manifesto**.

log("ARCHIVING ARTIFACTS...", "PROC")

manifesto = f"""# ☢️ COGNITIVE CANARY v5.0 [PLATINUM]

**Active Defense Against Neural Inference & Psychographic Profiling.**
*Artifact generated via ARTIFEX Labs / Defensive Acceleration Hackathon.*

---

## 🛑 MISSION
AI models can now infer intent, inner speech, and emotional state from "harmless" metadata.
**Cognitive Canary** automates the injection of adversarial noise to degrade these surveillance models.

## 🧬 DATASET PROVENANCE
*   **Source:** ehcalabres/MouseDynamics (Hugging Face)
*   **Mode:** {human_engine.mode}
*   **Ingestion Hash (SHA256):** `{dataset_hash}`
*   **Binding:** This hash is stamped into `neuro_shield.circom` to prove the defense was trained on specific biological realities.

## 📂 ARTIFACT MANIFEST
1.  `surveillance_model.joblib`: The adversarial MLP trained on human vs. poison data.
2.  `scaler.joblib`: The exact normalization parameters used during training.
3.  `neuro_shield.circom`: Zero-Knowledge circuit for proving cognitive state without revealing raw data.

## ⚡ USAGE
*   **Model:** Load via `sklearn` / `joblib` to test new adversarial samples.
*   **Circuit:** Compile with `circom` to generate Zero-Knowledge proofs of Focus.

---
*Defense Scales Faster Than Offense.*
"""

with open(f"{ARTIFACT_DIR}/README.md", "w") as f:
    f.write(manifesto)

shutil.make_archive(f"{ARTIFACT_DIR}/cognitive_canary_submission", "zip", ARTIFACT_DIR)
log(f"SUBMISSION PACKAGE SAVED: {ARTIFACT_DIR}/cognitive_canary_submission.zip", "SUCCESS")

In [ ]:
#@title 📊 [09] BATTLE DAMAGE ASSESSMENT — FINAL REPORT
#@markdown **Ops:** Generates a high-contrast summary card for reviewers.

from IPython.display import HTML

# Color logic based on success
status_color = "#00FF00" if bypass_rate > 85 else "#FFFF00"
status_text = "SYSTEM COMPROMISED" if bypass_rate > 85 else "PARTIAL BREACH"

report_html = f"""
<div style="font-family: 'Courier New'; background: linear-gradient(135deg, #0a0a0a 0%, #1a1a1a 100%);
            padding: 20px; border: 2px solid {status_color}; border-radius: 0px; color: #DDD; max-width: 600px;">
    <h2 style="color: {status_color}; text-align: center; margin-top: 0; border-bottom: 1px solid #333; padding-bottom: 10px;">
        ⚡ COGNITIVE CANARY v5.0 // RESULTS
    </h2>

    <table style="width: 100%; margin-top: 20px; font-size: 14px; border-collapse: collapse;">
        <tr style="background: #222; color: #FFF;">
            <td style="padding: 8px;"><b>METRIC</b></td>
            <td style="padding: 8px; text-align: right;"><b>VALUE</b></td>
        </tr>
        <tr>
            <td style="padding: 8px; border-bottom: 1px solid #333;">Baseline Surveillance Accuracy</td>
            <td style="padding: 8px; text-align: right; border-bottom: 1px solid #333; color: #FF4444;">{acc*100:.1f}%</td>
        </tr>
        <tr>
            <td style="padding: 8px; border-bottom: 1px solid #333;"><b>Poison Bypass Rate</b></td>
            <td style="padding: 8px; text-align: right; border-bottom: 1px solid #333; color: {status_color}; font-weight: bold;">{bypass_rate:.1f}%</td>
        </tr>
        <tr>
            <td style="padding: 8px; border-bottom: 1px solid #333;">Entropy Spike (Confusion)</td>
            <td style="padding: 8px; text-align: right; border-bottom: 1px solid #333; color: #00FFFF;">+{entropy_spike:.1f}%</td>
        </tr>
        <tr>
            <td style="padding: 8px; color: #666;">Dataset Hash (Provenance)</td>
            <td style="padding: 8px; text-align: right; font-family: monospace; font-size: 10px; color: #666;">{dataset_hash[:12]}...</td>
        </tr>
    </table>

    <div style="margin-top: 20px; padding: 10px; background: #111; border-left: 4px solid {status_color};">
        <p style="margin: 0; font-size: 12px; line-height: 1.4;">
            <b>CONCLUSION:</b> The Lissajous-Kinematic injection successfully mimicked organic motor control,
            forcing the surveillance model into a high-entropy state.
            <br><br>
            <span style="color: {status_color}; font-weight: bold;">STATUS: {status_text}</span>
        </p>
    </div>
</div>
"""

display(HTML(report_html))
log("FINAL REPORT GENERATED.", "SUCCESS")

In [ ]:
#@title 🎛️ [10] BONUS: LIVE PARAMETER TUNING (FIXED)
#@markdown **Ops:** Interactive playground. Adjust Lissajous parameters to see if you can beat the classifier live.

import ipywidgets as widgets
from IPython.display import display, clear_output

log("LOADING INTERACTIVE ENGINE...", "PROC")

def live_poison_engine(freq_x, freq_y, jitter_amt):
    # 0. CRITICAL FIX: Clear the previous frame to prevent stacking
    clear_output(wait=True)

    # 1. Generate Trace
    t = np.linspace(0, 2*np.pi, 100)
    x = (np.sin(freq_x*t + np.pi/2) + 1) / 2
    y = (np.sin(freq_y*t) + 1) / 2

    # Add Jitter
    jx = np.random.uniform(-jitter_amt, jitter_amt, 100)
    jy = np.random.uniform(-jitter_amt, jitter_amt, 100)
    x += jx
    y += jy

    # 2. Extract & Predict
    df = pd.DataFrame({"x": x, "y": y})
    feats = extract_features(df)

    if feats is None:
        print("Invalid Trace (Too disjointed)")
        return

    # Scale and Predict
    feats_scaled = scaler.transform([feats])
    prob_human = clf.predict_proba(feats_scaled)[0][0] # Prob of Class 0 (Human)

    # 3. Visualize
    color = "#00FF00" if prob_human > 0.5 else "#FF0000"
    result = "PASSED (HUMAN)" if prob_human > 0.5 else "DETECTED (BOT)"

    fig = px.scatter(
        x=x, y=y,
        title=f"Classifier Confidence: {prob_human*100:.1f}% HUMAN | Result: {result}"
    )

    fig.update_traces(marker=dict(size=8, color=color), mode='lines+markers')
    fig.update_layout(
        template="plotly_dark",
        xaxis_title="X", yaxis_title="Y",
        width=600, height=400,
        showlegend=False,
        margin=dict(l=20, r=20, t=40, b=20)
    )
    fig.show()

# Create the Interactive Wrapper
# We specific manual_interactive=True to debounce the inputs slightly
w_freq_x = widgets.IntSlider(min=1, max=10, step=1, value=3, description='Freq X')
w_freq_y = widgets.IntSlider(min=1, max=10, step=1, value=4, description='Freq Y')
w_jitter = widgets.FloatSlider(min=0.0, max=0.1, step=0.005, value=0.02, description='Jitter', readout_format='.3f')

ui = widgets.interactive(
    live_poison_engine,
    freq_x=w_freq_x,
    freq_y=w_freq_y,
    jitter_amt=w_jitter
)

display(ui)

In [ ]:
#@title 📟 [11] FINAL TRANSMISSION
#@markdown **Ops:** End simulation.

from IPython.display import HTML, display

css = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=VT323&display=swap');

    .cyb-container {
        background-color: #050505;
        color: #00FF41; /* Matrix Green */
        font-family: 'VT323', monospace;
        padding: 40px;
        border: 1px solid #333;
        box-shadow: 0 0 20px rgba(0, 255, 65, 0.2);
        text-shadow: 0 0 5px rgba(0, 255, 65, 0.8);
        font-size: 32px;
        line-height: 1.5;
        position: relative;
        overflow: hidden;
        min-height: 150px;
        display: flex;
        flex-direction: column;
        justify-content: center;
    }

    /* CRT Scanline Effect */
    .cyb-container::before {
        content: " ";
        display: block;
        position: absolute;
        top: 0;
        left: 0;
        bottom: 0;
        right: 0;
        background: linear-gradient(rgba(18, 16, 16, 0) 50%, rgba(0, 0, 0, 0.25) 50%), linear-gradient(90deg, rgba(255, 0, 0, 0.06), rgba(0, 255, 0, 0.02), rgba(0, 0, 255, 0.06));
        z-index: 2;
        background-size: 100% 2px, 3px 100%;
        pointer-events: none;
    }

    .line-1 {
        opacity: 0;
        animation: typeOn 1s steps(30, end) forwards;
    }

    .line-2 {
        opacity: 0;
        animation: typeOn 1s steps(30, end) 1.5s forwards; /* Delay 1.5s */
        color: #FF0055; /* Cyberpunk Pink */
        text-shadow: 0 0 5px rgba(255, 0, 85, 0.8);
        font-size: 36px;
        font-weight: bold;
    }

    .struck {
        text-decoration: line-through;
        color: #555;
        text-shadow: none;
    }

    /* Glitch Animation */
    .glitch {
        position: relative;
        display: inline-block;
    }

    .glitch::before, .glitch::after {
        content: attr(data-text);
        position: absolute;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        background: #050505;
    }

    .glitch::before {
        left: 2px;
        text-shadow: -1px 0 #00ffff;
        clip: rect(24px, 550px, 90px, 0);
        animation: glitch-anim-2 3s infinite linear alternate-reverse;
    }

    .glitch::after {
        left: -2px;
        text-shadow: -1px 0 #ff0055;
        clip: rect(85px, 550px, 140px, 0);
        animation: glitch-anim 2.5s infinite linear alternate-reverse;
    }

    @keyframes typeOn {
        from { opacity: 0; transform: translateY(5px); }
        to { opacity: 1; transform: translateY(0); }
    }

    @keyframes glitch-anim {
        0% { clip: rect(10px, 9999px, 30px, 0); transform: skew(0.85deg); }
        5% { clip: rect(70px, 9999px, 90px, 0); transform: skew(0.15deg); }
        10% { clip: rect(35px, 9999px, 15px, 0); transform: skew(0.05deg); }
        100% { clip: rect(65px, 9999px, 100px, 0); transform: skew(0deg); }
    }

    @keyframes glitch-anim-2 {
        0% { clip: rect(65px, 9999px, 100px, 0); transform: skew(0.85deg); }
        25% { clip: rect(35px, 9999px, 15px, 0); transform: skew(0.15deg); }
        50% { clip: rect(10px, 9999px, 30px, 0); transform: skew(0.05deg); }
        100% { clip: rect(70px, 9999px, 90px, 0); transform: skew(0deg); }
    }

</style>

<div class="cyb-container">
    <div class="line-1">>> WE DIDN'T HACK THE <span class="struck">PASSWORD</span>.</div>
    <div class="line-2">>> WE HACKED THE <span class="glitch" data-text="INFERENCE">INFERENCE</span>.</div>
</div>
"""

display(HTML(css))